# Importer les librairies, et fichiers .py auxiliaires

In [2]:
import numpy as np
from preprocess_data import *
from bayes_classifier import BayesClassifier
from complement_naive_bayes import train_cnb_with_tfidf, train_cnb
from ensemble_learning import train_ensemble
from xgboost_classifier import train_xgboost, train_xgboost_with_tfidf
from svc_classifier import *
from sgd_classifier import *
from sklearn.model_selection import StratifiedKFold

from save_output import save_output

In [3]:
data_preprocess = DataPreprocess()
X_train, y_train, X_test = data_preprocess.train, data_preprocess.label_train, data_preprocess.test

# Jalon 2)
# Naive Bayes Complement
K-Fold validation croisée stratifiée ($k = 5)$:
- α: Lissage du postérieur de Bayes
 
Prétraitement: TFIDF

In [7]:
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tfidf")

Fitting 2 folds for each of 50 candidates, totalling 100 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.111 total time=   1.5s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.110 total time=   1.5s
[CV 1/2] END .....cnb__alpha=1.4261714596148742;, score=0.003 total time=   1.5s
[CV 2/2] END .....cnb__alpha=1.4261714596148742;, score=0.007 total time=   1.5s
[CV 1/2] END .....cnb__alpha=1.0980909127171077;, score=0.005 total time=   1.5s
[CV 2/2] END .....cnb__alpha=1.0980909127171077;, score=0.019 total time=   1.5s
[CV 1/2] END .....cnb__alpha=0.8980877262955549;, score=0.016 total time=   1.5s
[CV 2/2] END .....cnb__alpha=0.8980877262955549;, score=0.026 total time=   1.5s
[CV 1/2] END ....cnb__alpha=0.23412796066365477;, score=0.448 total time=   1.5s
[CV 2/2] END ....cnb__alpha=0.23412796066365477;, score=0.420 total time=   1.5s
[CV 1/2] END ....cnb__alpha=0.23409178050430396;, score=0.448 total time=   1.5s
[CV 2/2] END ....cnb__alpha=0.23409178050430396

Prétraitement: Sans prétraitement

In [8]:
complement_naive_bayes = train_cnb(X_train, y_train)
predictions = complement_naive_bayes.predict(X_test)
save_output(predictions, "cnb", "random_search_50_iter", "no_preprocessing")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.588 total time=   0.5s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.575 total time=   0.5s
Best Parameters: {'cnb__alpha': 0.5619101782710437}
Best F1 Score: 0.581482890607508
predictions saved in output/20241111/223336_cnb_random_search_50_iter_no_preprocessing.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/204747_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/154833_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/162028_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/200251_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/162135_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/162421_cnb_random_search_50_iter_no_preprocessing.csv
	 output/20241111/154327_cnb_random_search_50_iter_no_preprocessing.csv
	 out

Prétraitement: TFIDF, réduction par arbre, stopwords retirés

In [4]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indices_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indices_features]
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(X_train, data_preprocess.label_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tree_reduction_stopwords_tfidf")

Fitting 2 folds for each of 50 candidates, totalling 100 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.572 total time=   0.1s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.556 total time=   0.1s
[CV 1/2] END .....cnb__alpha=1.4261714596148742;, score=0.568 total time=   0.1s
[CV 2/2] END .....cnb__alpha=1.4261714596148742;, score=0.544 total time=   0.1s
[CV 1/2] END .....cnb__alpha=1.0980909127171077;, score=0.572 total time=   0.1s
[CV 2/2] END .....cnb__alpha=1.0980909127171077;, score=0.553 total time=   0.1s
[CV 1/2] END .....cnb__alpha=0.8980877262955549;, score=0.571 total time=   0.1s
[CV 2/2] END .....cnb__alpha=0.8980877262955549;, score=0.554 total time=   0.1s
[CV 1/2] END ....cnb__alpha=0.23412796066365477;, score=0.562 total time=   0.1s
[CV 2/2] END ....cnb__alpha=0.23412796066365477;, score=0.551 total time=   0.1s
[CV 1/2] END ....cnb__alpha=0.23409178050430396;, score=0.562 total time=   0.1s
[CV 2/2] END ....cnb__alpha=0.23409178050430396

Prétraitement: TFIDF, réduction par arbre, stopwords retirés, suréchantillonnage SMOTE

In [5]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train, sorted_indices_features = tree_based_dimensionality_reduction(data_preprocess.train, data_preprocess.label_train)
X_test = data_preprocess.test[:, sorted_indices_features]
X_train, y_train = smote_oversampling(X_train, data_preprocess.label_train)

complement_naive_bayes = train_cnb(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)

save_output(predictions, "cnb", "random_search_50_iter", "tree_reduction_stopwords_tfidf_smote")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.635 total time=   0.0s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.655 total time=   0.0s
Best Parameters: {'cnb__alpha': 0.5619101782710437}
Best F1 Score: 0.6450953216247627
predictions saved in output/20241111/223814_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/200509_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/204814_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/215844_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/223408_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
Number of 0: 1419
Number of 1: 937
Ratio of 1: 0.3977079796264856
Number of differences with bayes classifier submission

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [6]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

complement_naive_bayes = train_cnb(X_train, y_train)
tfidf_test = tfidf_transformer.transform(X_test)
predictions = complement_naive_bayes.predict(tfidf_test)

save_output(predictions, "cnb", "random_search_50_iter", "stopwords_cum-sum_undersampled")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END .....cnb__alpha=0.5619101782710437;, score=0.635 total time=   0.0s
[CV 2/2] END .....cnb__alpha=0.5619101782710437;, score=0.655 total time=   0.0s
Best Parameters: {'cnb__alpha': 0.5619101782710437}
Best F1 Score: 0.6450953216247627
predictions saved in output/20241111/223815_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/200509_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/204814_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/223814_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/215844_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
	 output/20241111/223408_cnb_random_search_50_iter_tree_reduction_stopwords_tfidf_smote.csv
Number of 0: 1419
Number of 

# SVC
Grid Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- $\gamma$ : Coefficient du Noyau RBF
- $C$ : Terme de régularization pour la pénalité euclidienne.

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [7]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_params_, best_score_ = train_svc(X_train_undersampled, y_train_undersampled)
print(best_params_, best_score_)
svc = SVC(kernel='rbf', C=best_params_['C'], gamma=best_params_['gamma'])
svc.fit(X_train_undersampled, y_train_undersampled)
y_pred = svc.predict(data_preprocess.test)
params = f"C={best_params_['C']}, gamma={best_params_['gamma']}"
save_output(y_pred, "svm", params, "stopwords_cum-sum_undersampled")

{'gamma': array([0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01 , 0.011,
       0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019]), 'C': array([45, 46, 47, 48, 49, 50, 51, 52, 53, 54])}
Fitting 1 folds for each of 170 candidates, totalling 170 fits
[CV 1/1] END .................C=45, gamma=0.003;, score=0.635 total time=   0.5s
[CV 1/1] END .................C=45, gamma=0.004;, score=0.646 total time=   0.2s
[CV 1/1] END .................C=45, gamma=0.005;, score=0.651 total time=   0.2s
[CV 1/1] END .................C=45, gamma=0.006;, score=0.651 total time=   0.2s
[CV 1/1] END .................C=45, gamma=0.007;, score=0.650 total time=   0.2s
[CV 1/1] END .................C=45, gamma=0.008;, score=0.643 total time=   0.2s
[CV 1/1] END ..C=45, gamma=0.009000000000000001;, score=0.635 total time=   0.2s
[CV 1/1] END ..................C=45, gamma=0.01;, score=0.631 total time=   0.2s
[CV 1/1] END .................C=45, gamma=0.011;, score=0.620 total time=   0.2s
[CV 1/1] 

/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'C': 45, 'gamma': 0.006} 0.6510574018126888
predictions saved in output/20241111/224209_svm_C=45, gamma=0.006_stopwords_cum-sum_undersampled.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/163020_svm_C=45, gamma=0.006_stopwords_cum-sum_undersampled.csv
	 output/20241111/200600_svm_C=45, gamma=0.006_stopwords_cum-sum_undersampled.csv
	 output/20241111/162517_svm_C=45, gamma=0.006_stopwords_cum-sum_undersampled.csv
	 output/20241111/215926_svm_C=45, gamma=0.006_stopwords_cum-sum_undersampled.csv
	 output/20241111/204856_svm_C=45, gamma=0.006_stopwords_cum-sum_undersampled.csv
Number of 0: 1683
Number of 1: 673
Ratio of 1: 0.28565365025466893
Number of differences with bayes classifier submission: 396
Ratio of 1 in bayes submission: 0.29499151103565363


# SGDClassifier
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- loss: ModifiedHuber 
- penalty: ElaticNet 
- l1_ratio: Porportion de la perte d'ElasticNet qui est l1.
- $\alpha$: Poids attribué au terme de régularisation. Une plus grande valeur favorise que certains coefficients soient annulés (par l1) ou fortement adoucis (par l2) quand la pénalité est ElaticNet. 

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [8]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

best_sgd = train_sgd(X_train_undersampled, y_train_undersampled)
predictions_voter = best_sgd.predict(data_preprocess.test)
save_output(predictions_voter, "sgd", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[CV 1/2] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.562 total time=   1.3s
[CV 2/2] END model__alpha=0.027825594022071243, model__l1_ratio=0.667;, score=0.524 total time=   1.4s
[CV 1/2] END model__alpha=0.5994842503189409, model__l1_ratio=0.667;, score=0.000 total time=   0.4s
[CV 2/2] END model__alpha=0.5994842503189409, model__l1_ratio=0.667;, score=0.000 total time=   0.4s
[CV 1/2] END model__alpha=0.01, model__l1_ratio=0.223;, score=0.612 total time=   2.0s
[CV 2/2] END model__alpha=0.01, model__l1_ratio=0.223;, score=0.609 total time=   2.1s
[CV 1/2] END model__alpha=0.1291549665014884, model__l1_ratio=0.556;, score=0.198 total time=   0.7s
[CV 2/2] END model__alpha=0.1291549665014884, model__l1_ratio=0.556;, score=0.198 total time=   1.0s
[CV 1/2] END model__alpha=0.3593813663804626, model__l1_ratio=0.556;, score=0.000 total time=   0.4s
[CV 2/2] END model__alpha=0.3593813663804626, model__l

# XGBoost
Random Search, K-Fold validation croisée stratifiée ($k = 5)$, avec hyperparamètres:
- Learning rate: Taille du pas lors de la descente de critère sur la perte (gain de subdivision) 
- Nombre d'estimateurs (n_estimators): Nombre d'arbres de décision
- Profondeur maximale (max_depth): Profondeur maximale de chaque arbre (nombre de branchements max)
- sous-échantillon (subsample):  Proportion de données utilisées pour produire chaque arbre.

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [10]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

xgboost_classifier = train_xgboost(X_train_undersampled, y_train_undersampled)
predictions = xgboost_classifier.predict(data_preprocess.test)
save_output(predictions, "xgboost", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[CV 1/2] END model__learning_rate=0.0849080237694725, model__max_depth=3, model__n_estimators=400, model__subsample=0.892797576724562;, score=0.635 total time=   2.4s
[CV 2/2] END model__learning_rate=0.0849080237694725, model__max_depth=3, model__n_estimators=400, model__subsample=0.892797576724562;, score=0.618 total time=   2.4s
[CV 1/2] END model__learning_rate=0.12973169683940733, model__max_depth=7, model__n_estimators=300, model__subsample=0.662397808134481;, score=0.648 total time=   2.8s
[CV 2/2] END model__learning_rate=0.12973169683940733, model__max_depth=7, model__n_estimators=300, model__subsample=0.662397808134481;, score=0.642 total time=   3.0s
[CV 1/2] END model__learning_rate=0.021616722433639893, model__max_depth=10, model__n_estimators=200, model__subsample=0.8404460046972835;, score=0.615 total time=   2.9s
[CV 2/2] END model__learning_rate=0.021616722433639893, model__max_depth=10, model__n_estimators=2

# Apprentissage par ensembles: CNB, XGBoost, Logistic Regression

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [11]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled, y_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__alpha=0.3851694527491273;, score=0.689 total time=   0.1s
[CV 2/2] END ...model__alpha=0.3851694527491273;, score=0.678 total time=   0.1s
Best F1 for ComplementNB: 0.6837390151836877
Confusion Matrix - Fold 1:
[[1028  397]
 [ 227  692]]

Confusion Matrix - Fold 2:
[[1011  414]
 [ 235  684]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:48:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 1/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.625 total time=   1.3s


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:48:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.622 total time=   1.2s


...

Best F1 for XGBClassifier: 0.6321269577091664


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:49:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:49:57] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix - Fold 1:
[[1500  281]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1452  329]
 [ 462  687]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.581 total time=   0.1s
[CV 2/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.600 total time=   0.1s
Best F1 for LogisticRegression: 0.5907665275877441
Confusion Matrix - Fold 1:
[[1332  449]
 [ 494  655]]

Confusion Matrix - Fold 2:
[[1304  477]
 [ 452  697]]



/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:49:59] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


predictions saved in output/20241111/225002_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/155220_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
	 output/20241111/205843_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
Number of 0: 1569
Number of 1: 787
Ratio of 1: 0.3340407470288625
Number of differences with bayes classifier submission: 250
Ratio of 1 in bayes submission: 0.29499151103565363


Prétraitement: Retirer stopwords, sous-échantillonage aléatoire

In [13]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled, y_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords_undersampling")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__alpha=0.3851694527491273;, score=0.682 total time=   0.2s
[CV 2/2] END ...model__alpha=0.3851694527491273;, score=0.677 total time=   0.2s
Best F1 for ComplementNB: 0.6793106248810417
Confusion Matrix - Fold 1:
[[1023  402]
 [ 236  683]]

Confusion Matrix - Fold 2:
[[1019  406]
 [ 241  678]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:53:07] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 1/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.625 total time=   3.4s


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:53:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.622 total time=   3.3s


...

Best F1 for XGBClassifier: 0.6321269577091664


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:55:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:55:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix - Fold 1:
[[1500  281]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1452  329]
 [ 462  687]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.587 total time=   0.3s
[CV 2/2] END model__C=5.588135039273247, model__penalty=l1, model__solver=liblinear;, score=0.605 total time=   0.3s
Best F1 for LogisticRegression: 0.5961926125858565
Confusion Matrix - Fold 1:
[[1350  431]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1306  475]
 [ 447  702]]



/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:56:01] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


predictions saved in output/20241111/225608_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 output/20241111/210232_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling.csv
	 output/20241111/225304_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling.csv
	 output/20241111/155258_ensemble_cnb_xgboost_logreg_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
Number of 0: 1569
Number of 1: 787
Ratio of 1: 0.3340407470288625
Number of differences with bayes classifier submission: 236
Ratio of 1 in bayes submission: 0.29499151103565363


# Apprentissage par ensembles: Complement Naive Bayes, XGBoost, SVC, SGD

Prétraitement: Retirer stopwords, Réduction par somme cumulative, sous-échantillonage aléatoire

In [14]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
data_preprocess.remove_cum_sum()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []
model_names = ['ComplementNB', 'XGBoost', 'SVC', 'SGD']
# Validation croisée du voteur
for train_index, test_index in kf.split(X_train_undersampled, y_train_undersampled):
    # Split données
    X_train, X_test = X_train_undersampled[train_index], X_train_undersampled[test_index]
    y_train, y_test = y_train_undersampled[train_index], y_train_undersampled[test_index]
    best_ensemble_model = train_ensemble(X_train, y_train, model_names)    
    y_pred = best_ensemble_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    scores.append(score)

mean_score = np.mean(scores)
print("Score F1 de validation du voteur: ", mean_score)

best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_svc_sgd", "random_search_15_iter", "stopwords_undersampling_cumulative_sum")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__alpha=0.3851694527491273;, score=0.689 total time=   0.1s
[CV 2/2] END ...model__alpha=0.3851694527491273;, score=0.678 total time=   0.1s
Best F1 for ComplementNB: 0.6837390151836877
Confusion Matrix - Fold 1:
[[1028  397]
 [ 227  692]]

Confusion Matrix - Fold 2:
[[1011  414]
 [ 235  684]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:56:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 1/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.625 total time=   1.4s


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:56:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV 2/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.622 total time=   1.3s


...

Best F1 for XGBClassifier: 0.6321269577091664


/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:07:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:07:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Confusion Matrix - Fold 1:
[[1500  281]
 [ 492  657]]

Confusion Matrix - Fold 2:
[[1452  329]
 [ 462  687]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__C=47, model__gamma=0.016;, score=0.507 total time= 2.1min
[CV 2/2] END ...model__C=47, model__gamma=0.016;, score=0.522 total time= 2.1min
Best F1 for SVC: 0.5144048932843956
Confusion Matrix - Fold 1:
[[1630  151]
 [ 708  441]]

Confusion Matrix - Fold 2:
[[1549  232]
 [ 661  488]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END model__alpha=0.0774263682681127, model__l1_ratio=0.445;, score=0.441 total time=   0.8s
[CV 2/2] END model__alpha=0.0774263682681127, model__l1_ratio=0.445;, score=0.420 total time=   1.1s
Best F1 for SGDClassifier: 0.43032938318482
Confusion Matrix - Fold 1:
[[1599  182]
 [ 800  349]]

Confusion Matrix - Fold 2:
[[1553  228]
 [ 764  385]]



/home/ggenois/anaconda3/envs/rapids-24.08/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:22:04] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1722516643542/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


predictions saved in output/20241112/002919_ensemble_cnb_xgboost_svc_sgd_random_search_15_iter_stopwords_undersampling_cumulative_sum.csv
already same predictions saved in submissions? None
already same predictions saved in output?
	 None
Number of 0: 1570
Number of 1: 786
Ratio of 1: 0.333616298811545
Number of differences with bayes classifier submission: 171
Ratio of 1 in bayes submission: 0.29499151103565363
